<a href="https://colab.research.google.com/github/mr7495/image-classification-spatial/blob/main/Intel_DenseNet169_depthw_constaints_224.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi #show GPU type

In [2]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import zipfile
import shutil
import random
import pandas as pd
import csv
import os

In [ ]:
#download Intel image classification dataset from Kaggle (https://www.kaggle.com/puneet6060/intel-image-classification)
#Get a new download like from kaggle website at the mentioned URL, then replace the link with the "kaggle_linke" input in the next line of code 

!wget -cO - kaggle_link > intel.zip #replace kaggle_link with a new download link from https://www.kaggle.com/puneet6060/intel-image-classification

In [4]:
archive = zipfile.ZipFile('intel.zip') #Unzip Kaggle Dataset
for file in archive.namelist():
     archive.extract(file, 'data')

In [5]:
#Extract Training and Testing data
classes=['buildings','forest','glacier','mountain','sea','street']

train_data=[]
for r,d,f in os.walk('data/seg_train/seg_train'):
  for file in f:
    if '.jpg' in file:
      for clas in classes:
        if clas in r:
          train_data.append(['{}/{}'.format(r[5:],file),clas])
          break

test_data=[]
for r,d,f in os.walk('data/seg_test/seg_test'):
  for file in f:
    if '.jpg' in file:
      for clas in classes:
        if clas in r:
          test_data.append(['{}/{}'.format(r[5:],file),clas])
          break

In [6]:
#write train and test CSV files
with open('data/train.csv','w',newline='') as f:
  csvw=csv.writer(f)
  csvw.writerow(['filename','class'])
  for item in train_data:
    class_name=item[1]
    img_name=item[0]
    csvw.writerow([img_name,class_name])

with open('data/test.csv','w',newline='') as f:
  csvw=csv.writer(f)
  csvw.writerow(['filename','class'])
  for item in test_data:
    class_name=item[1]
    img_name=item[0]
    csvw.writerow([img_name,class_name])

In [7]:
#Set data augmentation techniques

train_datagen = keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True,vertical_flip=True
                                                             ,zoom_range=0.2,rotation_range=360
                                                             ,width_shift_range=0.1,height_shift_range=0.1
                                                             ,channel_shift_range=50
                                                             ,brightness_range=(0,1.2)
                                                             ,preprocessing_function=keras.applications.imagenet_utils.preprocess_input)

test_datagen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=keras.applications.imagenet_utils.preprocess_input)

train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

In [8]:
#Create Data augmentation techniques

batch_size=70
train_generator = train_datagen.flow_from_dataframe(
      dataframe=train_df,
      directory='data',
      x_col="filename",
      y_col="class",
      target_size=(224, 224),
      batch_size=batch_size,
      class_mode='categorical',shuffle=True)
validation_generator = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory='data',
        x_col="filename",
        y_col="class",
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical',shuffle=True)
train_img_num=len(train_generator.filenames)

Found 14034 validated image filenames belonging to 6 classes.
Found 3000 validated image filenames belonging to 6 classes.


In [ ]:
name="Intel-DenseNet169-depthw-constaints-224"
!mkdir "models" #create new folder for saving checkpoints
!mkdir "reports" #create new folder for saving evaluation reports
keras.backend.clear_session() #clear backend
shape=(224,224,3) 
input_tensor=keras.Input(shape=shape)
base_model=keras.applications.DenseNet169(input_tensor=input_tensor,weights=None,include_top=False)
depth=keras.layers.DepthwiseConv2D(tuple(base_model.output.shape[1:3]), depthwise_initializer=keras.initializers.RandomNormal(mean=0.0,stddev=0.01),
                                      bias_initializer=keras.initializers.Zeros(),depthwise_constraint=keras.constraints.NonNeg())(base_model.output)
flat=keras.layers.Flatten()(depth)
preds=keras.layers.Dense(6,activation='softmax',
                          kernel_initializer=keras.initializers.RandomNormal(mean=0.0,stddev=0.01),
                          bias_initializer=keras.initializers.Zeros(),)(flat)
model=keras.Model(inputs=base_model.input, outputs=preds)  

##################################
for layer in model.layers:
  layer.trainable = True
model.summary()
filepath="models/%s-{epoch:02d}-{val_accuracy:.4f}.hdf5"%name
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=False, mode='max') #creating checkpoint to save the best validation accuracy
callbacks_list = [checkpoint]

#Determine adaptive learning rate with an initialization value of 0.045 and decay of 0.94 every two epochs.
lr_schedule =keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.045,
    decay_steps=2*int(train_img_num/batch_size),
    decay_rate=0.94,
    staircase=True)
optimizer=keras.optimizers.SGD(momentum=0.9,learning_rate=lr_schedule)
model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])
hist=model.fit_generator(train_generator, epochs=148,validation_data=validation_generator,shuffle=True,callbacks=callbacks_list) #start training
with open('reports/{}.csv'.format(name), mode='w',newline='') as csv_file: #write reports
  csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  for key in hist.history:
    data=[key]
    data.extend(hist.history[key])
    csv_writer.writerow(data)
print("Training finished. Reports saved!")